<a href="https://colab.research.google.com/github/IvanSerem/ds2-proyecto-agricola/blob/main/DataScienceII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

“¿Cómo se relacionan las condiciones climáticas y las propiedades del suelo en un punto geográfico determinado?”

“Para enriquecer el modelo predictivo desarrollado en Data Science I, se incorporaron variables climáticas históricas obtenidas desde la NASA POWER API, una fuente científica abierta diseñada para aplicaciones agrícolas y ambientales.”

“Las variables climáticas externas se extrajeron para el mismo período temporal que el dataset base, garantizando coherencia temporal entre las fuentes.”

“Las variables climáticas fueron agregadas temporalmente para representar condiciones promedio/acumuladas relevantes para el crecimiento del cultivo.”

1.1. Importación de Librerías y Definición de Parámetros

“Este proyecto integra el modelo analítico desarrollado en Data Science I con datos climáticos y edáficos obtenidos mediante APIs externas, permitiendo analizar las condiciones reales que influyen en el rendimiento agrícola y sentando las bases para modelos predictivos futuros.”

“El proyecto de Data Science II extiende el modelo conceptual desarrollado en Data Science I, incorporando datos climáticos y edáficos reales obtenidos mediante APIs externas, demostrando la continuidad metodológica entre ambas etapas.”

In [ ]:
# Importamos las librerías esenciales
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

# Configuramos un estilo visual para todos nuestros gráficos
sns.set_theme(style="whitegrid")

# Definimos el endpoint base de SoilGrids (ISRIC)
url_base = "https://rest.isric.org/soilgrids/v2.0/properties/query"

1.2. Extracción de Datos en Lotes

In [1]:
# ============================================================
# SOILGRIDS (ISRIC) — Extracción en lotes por puntos (lat/lon)
# + Enriquecimiento con metadata (descripción + unidad)
# ============================================================

import pandas as pd
import requests
import time

# ----------------------------
# 1) Configuración API
# ----------------------------
url_base = "https://rest.isric.org/soilgrids/v2.0/properties/query"

# ----------------------------
# 2) Lote de puntos (recomendado: rurales/agro)
#    Podés reemplazar/expandir esta lista.
# ----------------------------
puntos_a_consultar = [
    {"id": "AR_Pergamino_Rural", "lat": -33.89, "lon": -60.57},
    {"id": "AR_Rosario_Rural",   "lat": -33.05, "lon": -60.65},
    {"id": "AR_Cordoba_Rural",   "lat": -31.20, "lon": -64.10},
    {"id": "AR_SantaFe_Rural",   "lat": -31.70, "lon": -60.70},
]

# ----------------------------
# 3) Parámetros SoilGrids (variables de suelo)
# ----------------------------
propiedades = ["phh2o", "soc", "clay"]      # pH, carbono orgánico, arcilla
profundidades = ["0-5cm", "5-15cm"]         # capas típicas
estadistico = "mean"                         # mean (o median si corresponde)

# ----------------------------
# 4) Robustez (reintentos)
# ----------------------------
TIMEOUT = 60
RETRIES = 3
SLEEP_BASE = 2  # backoff: 2^1=2s, 2^2=4s, 2^3=8s

# ----------------------------
# 5) Metadata (documentación) de variables
#    (para enriquecer el dataset final)
# ----------------------------
df_parametros_soilgrids = pd.DataFrame({
    "parameter": ["clay", "sand", "silt", "soc", "phh2o", "nitrogen", "cec", "bdod", "cfvo", "ocd"],
    "description": [
        "Porcentaje de arcilla",
        "Porcentaje de arena",
        "Porcentaje de limo",
        "Carbono orgánico del suelo",
        "pH del suelo (medido en agua)",
        "Contenido total de nitrógeno",
        "Capacidad de intercambio catiónico",
        "Densidad aparente",
        "Contenido de fragmentos gruesos",
        "Densidad de carbono orgánico"
    ],
    "unit": ["%", "%", "%", "g/kg", "pH", "g/kg", "cmol(+)/kg", "kg/m³", "%", "kg/m³"]
})

# ----------------------------
# 6) Extracción en lotes
# ----------------------------
lista_dataframes = []

print("--- Realizando la petición en lotes (SoilGrids por puntos) ---")

for p in puntos_a_consultar:
    print(f"\nObteniendo datos para: {p['id']} (lat={p['lat']}, lon={p['lon']})")

    params = {
        "lat": p["lat"],
        "lon": p["lon"],
        "property": propiedades,
        "depth": profundidades,
        "value": estadistico
    }

    data = None
    last_err = None

    for attempt in range(1, RETRIES + 1):
        try:
            response = requests.get(url_base, params=params, timeout=TIMEOUT)
            response.raise_for_status()
            data = response.json()
            break
        except requests.exceptions.RequestException as e:
            last_err = e
            wait = SLEEP_BASE ** attempt
            print(f"  ⚠️ intento {attempt}/{RETRIES} falló: {e} | reintento en {wait}s")
            time.sleep(wait)

    if data is None:
        print(f"❌ Error definitivo para {p['id']}: {last_err}")
        continue

    # ---- Parseo a formato "tidy": 1 fila = (punto, propiedad, profundidad)
    filas = []
    layers = data.get("properties", {}).get("layers", [])

    for item in layers:
        prop = item.get("name")  # ej: "phh2o"
        for d in item.get("depths", []):
            filas.append({
                "point_id": p["id"],
                "latitude": p["lat"],
                "longitude": p["lon"],
                "property": prop,
                "depth": d.get("label"),  # ej: "0-5cm"
                "value": d.get("values", {}).get(estadistico)
            })

    df_punto = pd.DataFrame(filas)

    total = len(df_punto)
    n_null = df_punto["value"].isna().sum() if total > 0 else 0
    print(f"✅ Registros obtenidos: {total} | nulls en value: {n_null}")

    lista_dataframes.append(df_punto)

# ----------------------------
# 7) Consolidación final (equivalente al 1.4)
# ----------------------------
if lista_dataframes:
    df_soilgrids_crudo = pd.concat(lista_dataframes, ignore_index=True)

    print("\n--- DataFrame Consolidado SoilGrids (CRUDO) ---")
    print("Shape:", df_soilgrids_crudo.shape)
    print("Columnas:", list(df_soilgrids_crudo.columns))
    display(df_soilgrids_crudo.head(10))

    # ----------------------------
    # 8) Enriquecimiento con metadata (description + unit)
    # ----------------------------
    df_soilgrids_enriquecido = df_soilgrids_crudo.merge(
        df_parametros_soilgrids,
        how="left",
        left_on="property",
        right_on="parameter"
    ).drop(columns=["parameter"])

    print("\n--- DataFrame SoilGrids (ENRIQUECIDO) ---")
    print("Shape:", df_soilgrids_enriquecido.shape)
    display(df_soilgrids_enriquecido.head(10))

    # ----------------------------
    # 9) Diagnósticos rápidos
    # ----------------------------
    print("\n--- Resumen de nulls (ENRIQUECIDO) ---")
    display(df_soilgrids_enriquecido.isna().sum())

    print("\n--- Propiedades solicitadas vs obtenidas ---")
    print("Solicitadas:", propiedades)
    print("Obtenidas  :", sorted(df_soilgrids_enriquecido["property"].dropna().unique().tolist()))

else:
    print("No se pudieron cargar datos de SoilGrids.")


--- Realizando la petición en lotes (SoilGrids por puntos) ---

Obteniendo datos para: AR_Pergamino_Rural (lat=-33.89, lon=-60.57)
✅ Registros obtenidos: 6 | nulls en value: 6

Obteniendo datos para: AR_Rosario_Rural (lat=-33.05, lon=-60.65)
✅ Registros obtenidos: 6 | nulls en value: 0

Obteniendo datos para: AR_Cordoba_Rural (lat=-31.2, lon=-64.1)
✅ Registros obtenidos: 6 | nulls en value: 0

Obteniendo datos para: AR_SantaFe_Rural (lat=-31.7, lon=-60.7)
✅ Registros obtenidos: 6 | nulls en value: 0

--- DataFrame Consolidado SoilGrids (CRUDO) ---
Shape: (24, 6)
Columnas: ['point_id', 'latitude', 'longitude', 'property', 'depth', 'value']


,point_id,latitude,longitude,property,depth,value
0,AR_Pergamino_Rural,-33.89,-60.57,clay,0-5cm,None
1,AR_Pergamino_Rural,-33.89,-60.57,clay,5-15cm,None
2,AR_Pergamino_Rural,-33.89,-60.57,phh2o,0-5cm,None
3,AR_Pergamino_Rural,-33.89,-60.57,phh2o,5-15cm,None
4,AR_Pergamino_Rural,-33.89,-60.57,soc,0-5cm,None
5,AR_Pergamino_Rural,-33.89,-60.57,soc,5-15cm,None
6,AR_Rosario_Rural,-33.05,-60.65,clay,0-5cm,282
7,AR_Rosario_Rural,-33.05,-60.65,clay,5-15cm,289
8,AR_Rosario_Rural,-33.05,-60.65,phh2o,0-5cm,62
9,AR_Rosario_Rural,-33.05,-60.65,phh2o,5-15cm,64



--- DataFrame SoilGrids (ENRIQUECIDO) ---
Shape: (24, 8)


,point_id,latitude,longitude,property,depth,value,description,unit
0,AR_Pergamino_Rural,-33.89,-60.57,clay,0-5cm,None,Porcentaje de arcilla,%
1,AR_Pergamino_Rural,-33.89,-60.57,clay,5-15cm,None,Porcentaje de arcilla,%
2,AR_Pergamino_Rural,-33.89,-60.57,phh2o,0-5cm,None,pH del suelo (medido en agua),pH
3,AR_Pergamino_Rural,-33.89,-60.57,phh2o,5-15cm,None,pH del suelo (medido en agua),pH
4,AR_Pergamino_Rural,-33.89,-60.57,soc,0-5cm,None,Carbono orgánico del suelo,g/kg
5,AR_Pergamino_Rural,-33.89,-60.57,soc,5-15cm,None,Carbono orgánico del suelo,g/kg
6,AR_Rosario_Rural,-33.05,-60.65,clay,0-5cm,282,Porcentaje de arcilla,%
7,AR_Rosario_Rural,-33.05,-60.65,clay,5-15cm,289,Porcentaje de arcilla,%
8,AR_Rosario_Rural,-33.05,-60.65,phh2o,0-5cm,62,pH del suelo (medido en agua),pH
9,AR_Rosario_Rural,-33.05,-60.65,phh2o,5-15cm,64,pH del suelo (medido en agua),pH



--- Resumen de nulls (ENRIQUECIDO) ---


,0
point_id,0
latitude,0
longitude,0
property,0
depth,0
value,6
description,0
unit,0



--- Propiedades solicitadas vs obtenidas ---
Solicitadas: ['phh2o', 'soc', 'clay']
Obtenidas  : ['clay', 'phh2o', 'soc']


1.3. Inspeccionando los Resultados del Bucle

1.4. Consolidación y Verificación Finalmente, unimos todos los DataFrames de nuestra lista en una única tabla maestra con pd.concat y realizamos una primera verificación.